# Digital-Contact-Tracing on SocioPattern

This notebook demonstrates the use of the simulator on a SocioPattern dataset.

In [1]:
import numpy as np
import DigitalContactTracing as DCT
from load_temporal_graph_tmp import load_df_socio, build_graphs_socio, get_array_of_contacts

%load_ext autoreload
%autoreload 2

### Imports and setup

In [2]:
PARAMETERS = {"temporal_gap" : 300,                 # Temporal gap between static networks (sec)
              "memory_contacts" : 7,                # Tracing memory (days)
              "max_time_quar" : 10,                 # Quarantine duration (days)
              "policies" :                          # Digital tracing policies [duration (sec)]
              {'policy1': [30*60],
               'policy2': [20*60],
               'policy3': [15*60],
               'policy4': [10*60],
               'policy5': [5*60]},  
              "eps_Is" : [0.2, 0.5, 0.8, 1],        # Isolation effectivity
              "times" : 10,                         # Number of repetition of the simulation
              "seed" : 11,                          # Random seed 
              "Y_i" : 1,                            # Number of initially infected people
              "nc" : 0.2,                           # Fraction of individuals who are not adopting the app
              "beta_t": 0.002,                      # Parameter defining the infectiousness probability
              "symptomatics" : 0.8,                 # Fraction of symptomatic individuals 
              "testing" : 0.0,                      # Fraction of asymptomatics who are detected via random testing
              "store":{"to_store": True,            # Save the results?
                       "path_to_store":"RESULTS_Socio/"}} # Target folder to save the results

### Define the parameters of the simulation

In [3]:
df = load_df_socio("Dataset/High-School.csv")
graphs = build_graphs_socio(get_array_of_contacts(df, PARAMETERS["temporal_gap"], column_name='time'),
                            PARAMETERS["temporal_gap"])

### Load the temporal graph

In [4]:
# Initialize the random seed
np.random.seed(PARAMETERS["seed"])

# Run over each configuration
for eps_I in PARAMETERS["eps_Is"]:
    for policy in PARAMETERS["policies"]:
        filter_duration = PARAMETERS["policies"][policy][0]
        filter_rssi=None
        results = []
        
        # Print some report
        print("-" * 100)
        print('Running simulation with policy: %s' % policy)
        print('       |_ eps_I               : %+4.2d'   % eps_I)
        print('       |_ filter_duration     : %+4.2d' % filter_duration)
        print('')
        
        # Repeat the simulation for a fixed configuration 
        for i in range(PARAMETERS["times"]):
            # Print some report
            print('       Running simulation     [%4d / %4d]' % (i + 1, PARAMETERS['times']))

            # Initialize the simulation object
            dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration,
                                            use_rssi=False)
            # Run the actual simulation
            res = dct.simulate()
            
            # Save the results    
            DCT.store_real_time(res,PARAMETERS,filter_rssi,filter_duration,eps_I)        

----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I               :  +00
       |_ filter_duration     : +1800

       Running simulation     [   1 /   10]
       Running simulation     [   2 /   10]
       Running simulation     [   3 /   10]
       Running simulation     [   4 /   10]
       Running simulation     [   5 /   10]


KeyboardInterrupt: 